In [0]:
%tensorflow_version 2.x

from tensorflow import math as mt
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


import tensorflow as tf
import keras
from keras.optimizers import Adam

from google.colab import drive
drive.mount("/content/gdrive/")

TensorFlow 2.x selected.
Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


Using TensorFlow backend.


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/AutoML/CNN/kettle')

from x_pipeline import XPipeline
from y_pipeline import YPipeline

In [0]:
X_train = pd.read_csv(
    'gdrive/My Drive/AutoML/data/X_train_6GWGSxz.csv',
)
X_train.set_index("time_step", inplace=True)
Y_train = pd.read_csv(
    'gdrive/My Drive/AutoML/data/y_train_2G60rOL.csv',
)
Y_train.set_index("time_step", inplace=True)

px = XPipeline()
py = YPipeline()

In [0]:
X_train.columns

Index(['consumption', 'visibility', 'temperature', 'humidity', 'humidex',
       'windchill', 'wind', 'pressure', 'Unnamed: 9'],
      dtype='object')

In [0]:
pd.set_option('display.max_columns', None)

## Preprocessing

In [0]:
print('Start of first transform')
x = px.fit(X_train)
x = px.transform(X_train)
print('End of first transform')
y = py.fit(Y_train)
y = py.transform(Y_train)
print('Second transform')
x_train, y_train = x[:321000, :, :], y[:321000]
x_valid, y_valid = x[321000:, :, :], y[321000:]

Start of first transform
End of first transform
Second transform


In [0]:
x.shape

(401759, 5, 1)

In [0]:
print(f"x_train shape is {x_train.shape}")
print(f"x_valid shape is {x_valid.shape}")

x_train shape is (321000, 5, 1)
x_valid shape is (80759, 5, 1)


## Wave Net

In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer(input_shape=[5, 1]))

model.add(tf.keras.layers.Conv1D(filters=20, kernel_size=2, padding="same",
                              activation="relu"))
model.add(tf.keras.layers.Conv1D(filters=20, kernel_size=2, padding="same",
                              activation="relu"))
model.add(tf.keras.layers.Conv1D(filters=30, kernel_size=2, padding="same",
                              activation="relu"))
model.add(tf.keras.layers.Conv1D(filters=30, kernel_size=2, padding="same",
                              activation="relu"))
# model.add(tf.keras.layers.BatchNormalization())
# model.add(tf.keras.layers.AveragePooling1D(pool_size=2, strides=None, padding='valid'))


model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='linear'))

In [0]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 19, 20)            60        
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 18, 20)            820       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 9, 20)             0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 8, 40)             1640      
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 7, 50)             4050      
_________________________________________________________________
flatten_3 (Flatten)          (None, 350)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 20)               

Metric: Mean Squared Error. Same as minimizing the Metric on the platform.

In [0]:
model.compile(loss=keras.losses.mean_squared_error, optimizer=tf.keras.optimizers.Adam(lr=0.001))
history = model.fit(x_train, y_train, epochs=4,
                    validation_data=(x_valid, y_valid))

Train on 321000 samples, validate on 80759 samples
Epoch 1/4
321000/321000 [==============================] - 38s 119us/sample - loss: 5288.1246 - val_loss: 11733.1721
Epoch 2/4
321000/321000 [==============================] - 37s 117us/sample - loss: 4883.3181 - val_loss: 14425.4797
Epoch 3/4
 10752/321000 [>.............................] - ETA: 31s - loss: 3385.7019

In [0]:
pred_val = model.predict(x_valid)

### Evaluating Performance

In [0]:
pred = pd.DataFrame(pred_val, columns=["kettle"])

In [0]:
(y_train>1000).sum()

In [0]:
pred.kettle.value_counts()

In [0]:
def nilm_metric(y_true, y_pred):
        score = 0.0
        score += math.sqrt(sum((y_pred - y_true) ** 2) / len(y_true)) * 14.57
        score /= 74.86
        return score

In [0]:
nilm_metric(y_valid, pred.values)

In [0]:
plt.figure(num=None, figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
plt.scatter(pred, y_valid)
plt.plot(np.linspace(0,250), np.linspace(0,250), c="orange")
plt.show()

### submission

In [0]:
X_test = pd.read_csv(
    '../provided_data_and_metric/X_test_c2uBt2s.csv',
)
time = X_test["time_step"]
X_test.set_index("time_step", inplace=True)

In [0]:
x_test = px.transform(X_test)

In [0]:
pred = model.predict(x_test)
pred = pd.DataFrame(pred, columns=COLUMNS)

In [0]:
pred = pd.concat([time, pred], axis=1)

In [0]:
# pred.fillna(method="ffill", inplace=True)

In [0]:
pred.to_csv("test_submission.csv", index=False)